In [2]:
import pandas as pd
import numpy as np

# Read the course_master.csv file
df = pd.read_csv("course_master.csv")
n = len(df)

# Randomly shuffle the indices for later assignment of exam durations
indices = df.index.to_list()
np.random.shuffle(indices)

# Calculate the counts for each exam duration
count_180 = int(0.7 * n)
count_120 = int(0.2 * n)
count_90  = n - count_180 - count_120

# Assign exam duration values based on the computed counts
df.loc[indices[:count_180], "Exam Duration"] = 180
df.loc[indices[count_180:count_180+count_120], "Exam Duration"] = 120
df.loc[indices[count_180+count_120:], "Exam Duration"] = 90

# Define the days and times lists
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri','Sat']
times = ['08:00', '08:15', '08:30', '08:45',
         '09:00', '09:15', '09:30', '09:45',
         '10:00', '10:15', '10:30', '10:45',
         '11:00', '11:15', '11:30', '11:45',
         '12:00', '12:15', '12:30', '12:45', '13:00',
         '14:00', '14:15', '14:30', '14:45',
         '15:00', '15:15', '15:30', '15:45',
         '16:00', '16:15', '16:30', '16:45',
         '17:00', '17:15', '17:30', '17:45', '18:00']

lunch_time_index = times.index('13:00')
pre_lunch_times = list(range(lunch_time_index + 1))
post_lunch_times = list(range((lunch_time_index + 1), len(times)))

def begin_end_pairs(session_len:int) -> list:
    pairs = []
    for s in range(0, len(times) - session_len):
        e = s + session_len
        if (((s in pre_lunch_times) and (e in pre_lunch_times)) or
            ((s in post_lunch_times) and (e in post_lunch_times))):
            pairs.append((s, e))
    return pairs

# Create a helper function to convert time string to minutes since midnight.
def time_to_minutes(time_str):
    hh, mm = map(int, time_str.split(":"))
    return hh * 60 + mm

# Now build the "Preferred Timings" column.
# For each course, determine session_len from its exam duration (assuming durations are in multiples of 15),
# then get the candidate begin-end pairs and combine each with each day.
# Finally, randomly sample a few candidate triplets (here sample_size=3 per course).

sample_size = 3
preferred_timings = []

for idx, row in df.iterrows():
    duration = int(row["Exam Duration"])  # duration in minutes
    session_len = duration // 15           # session length in terms of time slots (15-min intervals)
    pairs = begin_end_pairs(session_len)
    
    candidate_triplets = []
    # Form candidate triplets: convert slot indices to actual time strings and combine with a day
    for s, e in pairs:
        begin_time = times[s]
        end_time = times[e]
        # Check that the difference equals the duration (for sanity)
        if time_to_minutes(end_time) - time_to_minutes(begin_time) == duration:
            for day in days:
                candidate_triplets.append((begin_time, end_time, day))
                
    # Randomly sample sample_size triplets (or fewer if not enough candidates)
    if candidate_triplets:
        indices_sample = np.random.choice(len(candidate_triplets), size=min(sample_size, len(candidate_triplets)), replace=False)
        sampled_triplets = [candidate_triplets[i] for i in indices_sample]
    else:
        sampled_triplets = []
    
    preferred_timings.append(sampled_triplets)

df["Preferred Timings"] = preferred_timings

# Save the updated dataframe back to course_master.csv
df.to_csv("course_master.csv", index=False)